In [1]:
import pandas as pd
df=pd.read_excel("../my_data/tny.xlsx")
df = df.rename(columns={'winner':'w_name','loser':'l_name','tourney_year_id':'tny_id',\
                           'tourney_name_x':'tny_name','start_date':'tny_date','match_index':'match_num'})
df['match_num'] = df['match_num'].astype(str)
df['match_num'] = [s.replace('MS','') for s in df['match_num']]
df['tny_date'] = df['tny_date'].astype(str)
df['tny_date'] = [s.replace('.','') for s in df['tny_date']]

In [2]:
df['tny_name'] = ['Davis Cup' if 'Davis Cup' in s else s for s in df['tny_name']]
df['tny_name'] = [s.replace('Australian Chps.','Australian Open').replace('Australian Open-2',\
            'Australian Open').replace('U.S. National Chps.','US Open') for s in df['tny_name']]

In [3]:
import datetime
import re
def normalize_name(s,tour='atp'):
    if tour=='atp':
        s = s.replace('-',' ')
        s = s.replace('Stanislas','Stan').replace('Stan','Stanislas')
        s = s.replace('Alexandre','Alexander')
        s = s.replace('Federico Delbonis','Federico Del').replace('Federico Del','Federico Delbonis')
        s = s.replace('Mello','Melo')
        s = s.replace('Cedric','Cedrik')
        s = s.replace('Bernakis','Berankis')
        s = s.replace('Hansescu','Hanescu')
        s = s.replace('Teimuraz','Teymuraz')
        s = s.replace('Vikor','Viktor')
        s = s.rstrip()
        s = s.replace('Alex Jr.','Alex Bogomolov')
        s = s.title()
        sep = s.split(' ')
        return ' '.join(sep[:2]) if len(sep)>2 else s
    else:
        return s


TOUR='atp'
df['w_name'] = [normalize_name(x,tour=TOUR) for x in df['w_name']]
df['l_name'] = [normalize_name(x,tour=TOUR) for x in df['l_name']]
df['tny_name'] = ['Davis Cup' if 'Davis Cup' in s else s for s in df['tny_name']]
df['tny_name'] = [s.replace('Australian Chps.','Australian Open').replace('Australian Open-2',\
	            'Australian Open').replace('U.S. National Chps.','US Open') for s in df['tny_name']]

def format_match_df(df,ret_strings=[],abd_strings=[]):
    #df['pbp'] = [None]*len(df)
    df = df[~df['tny_date'].isin(['nan'])]
    #df = df.dropna(subset=['tny_date'])
    df = df.rename(columns={'winner_name':'w_name','loser_name':'l_name'})
    grand_slam_d = dict(zip(['Australian Open','Roland Garros','Wimbledon','US Open'],[1]*4))
    df['is_gs'] = [name in grand_slam_d for name in df['tny_name']]

    # Get dates into the same format
    df['tny_date'] = [datetime.datetime.strptime(str(x), "%Y%m%d").date() for x in df['tny_date']]
    df['match_year'] = [x.year for x in df['tny_date']]
    df['match_month'] = [x.month for x in df['tny_date']]
    df['score'] = [re.sub(r"[\(\[].*?[\)\]]", "", str(s)) for s in df['score']] # str(s) fixes any nans
    df['score'] = ['RET' if 'RET' in s else s for s in df['score']]
    df['w_swon'] = df['winner_first_serve_points_won'] + df['winner_second_serve_points_won'] 
    df['l_swon'] = df['loser_first_serve_points_won'] + df['loser_second_serve_points_won']
    df['w_rwon'] = df['winner_first_serve_return_won'] + df['winner_second_serve_return_won']
    df['l_rwon'] = df['loser_first_serve_return_won'] + df['loser_second_serve_return_won']
    df['w_rpt'] = df['winner_first_serve_return_total'] + df['winner_second_serve_return_total']
    df['l_rpt'] = df['loser_first_serve_return_total'] + df['loser_second_serve_return_total']

    # get rid of leading 0s in tny_id
    df['tny_id'] = ['-'.join(t.split('-')[:-1] + [t.split('-')[-1][1:]]) if t.split('-')[-1][0]=='0' else t \
                                    for t in df['tny_id']]

    # get rid of matches involving a retirement
    #df['score'] = ['ABN' if score.split(' ')[-1] in ('abandoned','ABN','ABD','DEF','def','unfinished','Walkover') \
    #                            else score for score in df['score']]
    #ret_strings = ['ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan','walkover']
    #ret_d = dict(zip(ret_strings,[1]*len(ret_strings)))
    #df = df.loc[[i for i in range(len(df)) if df['score'][i] not in ret_d]]
    df = df.sort_values(by=['tny_date','tny_name','match_num'], ascending=True).reset_index()
    del df['index']; return df

In [4]:
ret_strings = ('ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan','walkover')
abd_strings = ('abandoned','ABN','ABD','DEF','def','unfinished','Walkover')
atp_all_matches = format_match_df(df,ret_strings=ret_strings,abd_strings=abd_strings)
atp_all_matches = atp_all_matches[['tny_id','tny_name','tny_date','match_num','surface','is_gs','w_name','l_name',
                'winner_player_id','loser_player_id','match_year','match_month','score']]

In [5]:
print atp_all_matches.shape
atp_all_matches.head()

(74886, 13)


,tny_id,tny_name,tny_date,match_num,surface,is_gs,w_name,l_name,winner_player_id,loser_player_id,match_year,match_month,score
0,2001-7308,Adelaide,2001-01-01,001,Hard,False,Tommy Haas,Nicolas Massu,h355,m655,2001,1,6-3 6-1
1,2001-7308,Adelaide,2001-01-01,002,Hard,False,Tommy Haas,Jason Stoltenberg,h355,s331,2001,1,7-6 6-4
2,2001-7308,Adelaide,2001-01-01,003,Hard,False,Nicolas Massu,Tim Henman,m655,h336,2001,1,3-6 7-5 6-2
3,2001-7308,Adelaide,2001-01-01,004,Hard,False,Tommy Haas,Lleyton Hewitt,h355,h432,2001,1,6-4 0-6 6-1
4,2001-7308,Adelaide,2001-01-01,005,Hard,False,Jason Stoltenberg,Alberto Martin,s331,m590,2001,1,6-3 6-4


In [6]:
tnynametest = atp_all_matches
grandslam = 0.0
tourfinals = 0.0
masters = 0.0
olympics = 0.0
othergames = 0.0
for i in range(len(tnynametest)):
    if "Grand Slam" in tnynametest['tny_name'][i]:
        grandslam = grandslam + 1.0
    elif "Tour Finals" in tnynametest['tny_name'][i]:
        tourfinals = tourfinals + 1.0
    elif "Masters" in tnynametest['tny_name'][i]:
        masters = masters + 1.0
    elif "Olympics" in tnynametest['tny_name'][i]:
        olympics = olympics + 1.0
    else:
        othergames = othergames + 1.0
        
print grandslam, tourfinals, masters, olympics, othergames

0.0 0.0 14599.0 132.0 60155.0


## Split the training and test data

In [7]:
c = 2015
train_df,test_df = atp_all_matches[atp_all_matches['match_year']<=c],atp_all_matches[atp_all_matches['match_year']>c]
print len(train_df)
print len(test_df)

58502
16384


In [8]:
import numpy as np
import elo_538_jy as elo
COUNT = 0
players_list = np.union1d(train_df.w_name, train_df.l_name)
players_elo = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))]))
surface_elo = {}
for surface in ('Hard','Clay','Grass'):
    surface_elo[surface] = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))])) 

elo_1s, elo_2s = [],[]
surface_elo_1s, surface_elo_2s = [],[]
elo_obj = elo.Elo_Rater()

for i, row in train_df.iterrows():
    surface = row['surface']; is_gs = row['is_gs']
    # append elos, rate, update
    w_elo,l_elo = players_elo[row['w_name']],players_elo[row['l_name']]
    elo_1s.append(w_elo.value);elo_2s.append(l_elo.value)    
    
    elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT)

    
    surface_elo_1s.append(surface_elo[surface][row['w_name']].value if surface in ('Hard','Clay','Grass') else w_elo.value)
    surface_elo_2s.append(surface_elo[surface][row['l_name']].value if surface in ('Hard','Clay','Grass') else l_elo.value)
    if surface in ('Hard','Clay','Grass'):
        new_elo1, new_elo2 = elo_obj.rate_1vs1(surface_elo[surface][row['w_name']],surface_elo[surface][row['l_name']],is_gs,counts=COUNT)

# add columns
train_df['w_elo'], train_df['l_elo'] = elo_1s, elo_2s
train_df['sw_elo'], train_df['sl_elo'] = surface_elo_1s, surface_elo_2s

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
COUNT = 1
players_list = np.union1d(train_df.w_name, train_df.l_name)
players_elo = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))]))
surface_elo = {}
for surface in ('Hard','Clay','Grass'):
    surface_elo[surface] = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))])) 

elo_1s, elo_2s = [],[]
surface_elo_1s, surface_elo_2s = [],[]
elo_obj = elo.Elo_Rater()

for i, row in train_df.iterrows():
    surface = row['surface']; is_gs = row['is_gs']
    # append elos, rate, update
    w_elo,l_elo = players_elo[row['w_name']],players_elo[row['l_name']]
    elo_1s.append(w_elo.value);elo_2s.append(l_elo.value)    
    
    elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT)

    
    surface_elo_1s.append(surface_elo[surface][row['w_name']].value if surface in ('Hard','Clay','Grass') else w_elo.value)
    surface_elo_2s.append(surface_elo[surface][row['l_name']].value if surface in ('Hard','Clay','Grass') else l_elo.value)
    if surface in ('Hard','Clay','Grass'):
        new_elo1, new_elo2 = elo_obj.rate_1vs1(surface_elo[surface][row['w_name']],surface_elo[surface][row['l_name']],is_gs,counts=COUNT)

# add columns
train_df['w_elo_538'], train_df['l_elo_538'] = elo_1s, elo_2s
train_df['w_sf_elo_538'], train_df['l_sf_elo_538'] = surface_elo_1s, surface_elo_2s

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
#rankings
rankings = sorted(zip(players_elo.keys(),players_elo.values()),key=lambda x: x[1].value,reverse=True)
rankings

[(u'Andre Agassi', elo_538_jy.Rating(1503.115, times=322)),
 (u'Lleyton Hewitt', elo_538_jy.Rating(1503.058, times=702)),
 (u'Roger Federer', elo_538_jy.Rating(1502.576, times=1670)),
 (u'Andy Roddick', elo_538_jy.Rating(1502.343, times=804)),
 (u'Rafael Nadal', elo_538_jy.Rating(1502.218, times=1351)),
 (u'Juan Carlos', elo_538_jy.Rating(1502.214, times=620)),
 (u'Gustavo Kuerten', elo_538_jy.Rating(1502.036, times=227)),
 (u'Tim Henman', elo_538_jy.Rating(1501.911, times=363)),
 (u'Tommy Haas', elo_538_jy.Rating(1501.763, times=644)),
 (u'Carlos Moya', elo_538_jy.Rating(1501.754, times=533)),
 (u'Jo Wilfried', elo_538_jy.Rating(1501.675, times=589)),
 (u'Guillermo Coria', elo_538_jy.Rating(1501.643, times=323)),
 (u'Marat Safin', elo_538_jy.Rating(1501.625, times=459)),
 (u'Guillermo Canas', elo_538_jy.Rating(1501.602, times=369)),
 (u'Andy Murray', elo_538_jy.Rating(1501.534, times=886)),
 (u'Novak Djokovic', elo_538_jy.Rating(1501.521, times=1222)),
 (u'David Nalbandian', elo_538_j

In [11]:
list1 = []
list2 = []
for i in range(len(test_df)):
    
    name = test_df['w_name'][i+58502]
    if name in players_elo.keys():
        if name not in list1:
            list1.append(name)
    else:
        if name not in list2:
            list2.append(name)

print len(list1)
print len(list2)
list1

# for s in list1:
#     print players_elo[s]

df_test1 = test_df

for s in list2:
    df_test1 = df_test1[df_test1.w_name != s]
    df_test1 = df_test1[df_test1.l_name != s]
    
df_test2 = df_test1
df_test2.reset_index(drop=True,inplace=True)
df_test2

sumofright = 0.0
for i in range(len(df_test2)):
    if df_test2['w_name'][i] in players_elo and df_test2['l_name'][i] in players_elo:
        if players_elo[df_test2['w_name'][i]].value - players_elo[df_test2['l_name'][i]].value > 0:
            sumofright = sumofright + 1


sumofright/len(df_test2)

436
100


0.5696605374823197